In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install evaluate>=0.30
!pip install jiwer
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
json_data['RecordingMetadata']['prompt']
json_data['SpeakerMetadata']['topik_level']

In [ ]:
# 1. 오디오 파일 경로 취합
import glob

path = "/content/drive/MyDrive/STTDATA/데이터모음/원본데이터/*"
raw_data_list = glob.glob(path)
raw_data_list = sorted(raw_data_list)
print(f"file_list : {raw_data_list[:10]}")
print(len(raw_data_list))

In [ ]:
# 2. json 파일 경로 취합
import glob

path = "/content/drive/MyDrive/STTDATA/데이터모음/정답데이터/*"
labeled_data_list = glob.glob(path)
labeled_data_list = sorted(labeled_data_list)
print(f"file_list : {labeled_data_list[:10]}")
print(len(labeled_data_list))

In [ ]:
from tqdm import tqdm
import json
import pandas as pd

transcript_list = []
for labeled_data in tqdm(labeled_data_list):
    with open(labeled_data, "r", encoding="utf-8") as f:
        json_data = json.load(f)
        json_data = json_data['RecordingMetadata']['orthographic']
        transcript_list.append(json_data)

df = pd.DataFrame(data=transcript_list, columns = ["transcript"])

# 텍스트 데이터로 만든 데이터프레임에 음성 파일 경로 컬럼을 추가
df["raw_data"] = raw_data_list

# Null data 유무 확인
df.isnull().values.sum()

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import Dataset, DatasetDict
from datasets import Audio

# 오디오 파일 경로를 dict의 "audio" 키의 value로 넣고 이를 데이터셋으로 변환
# 이때, Whisper가 요구하는 사양대로 Sampling rate는 16,000으로 설정한다.
ds = Dataset.from_dict({"audio": [path for path in df["raw_data"]],
                       "transcripts": [transcript for transcript in df["transcript"]]}).cast_column("audio", Audio(sampling_rate=16000))

# 데이터셋을 훈련 데이터와 테스트 데이터, 밸리데이션 데이터로 분할
train_testvalid = ds.train_test_split(test_size=0.2)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})

# 작성한 데이터셋을 허깅페이스에 업로드
datasets.push_to_hub("jwh1449/AIhub_foreign_dataset")
print("완료")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 1. 오디오 파일 경로 취합
import glob

path = "/content/drive/MyDrive/STTDATA/데이터모음/원본데이터/*"
raw_data_list = glob.glob(path)
raw_data_list = sorted(raw_data_list)
print(f"file_list : {raw_data_list[:10]}")
print(len(raw_data_list))

In [ ]:
# 2. json 파일 경로 취합
import glob

path = "/content/drive/MyDrive/STTDATA/데이터모음/정답데이터/*"
labeled_data_list = glob.glob(path)
labeled_data_list = sorted(labeled_data_list)
print(f"file_list : {labeled_data_list[:10]}")
print(len(labeled_data_list))

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer

# 파인튜닝을 진행하고자 하는 모델의 feature extractor를 로드
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

# 파인튜닝을 진행하고자 하는 모델의 tokenizer를 로드
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")

In [ ]:
from datasets import load_dataset

# 전체 데이터셋에서 각 split의 50%만 불러오기
low_call_voices = load_dataset(
    "jwh1449/AIhub_foreign_dataset",
    split={
        "train": "train[:50%]",
        "test": "test[:50%]",
        "valid": "valid[:50%]"
    }
)

low_call_voices.push_to_hub("jwh1449/AIhub_foreign_dataset2")
print("완료")

In [ ]:
from datasets import load_dataset

# 전체 데이터셋에서 각 split의 50%만 불러오기
low_call_voices = load_dataset(
    "jwh1449/AIhub_foreign_dataset",
    split={
        "train": "train[-50%:]",
        "test": "test[-50%:]",
        "valid": "valid[-50%:]"
    }
)

low_call_voices.push_to_hub("jwh1449/AIhub_foreign_dataset3")
print("완료")

In [ ]:
from datasets import load_dataset

# 데이터셋을 업로드할 때 접근을 제한하거나 비공개로 설정한 경우 허깅페이스 로그인이 필요하다.
low_call_voices = load_dataset("jwh1449/AIhub_foreign_dataset3")

def prepare_dataset(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    # input audio array로부터 log-Mel spectrogram 변환
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # target text를 label ids로 변환
    batch["labels"] = tokenizer(batch["transcripts"]).input_ids
    return batch